# Main Dashboard of PhD experiments in pre silicon full

In [5]:
results_file = 'presilicon_results.csv'
import os
from pca.analytics import *
from pre_silicon.presi_data_gen import *
from clustering.distortion import *
from clustering.clustering import *
from clustering.cluster_score import *

# Characterization of worklaods based on instructions kind

In [7]:
files = dir_list = os.listdir("tests/pre_silicon_runs_ml_db")
my_labels = ['arithmetic', 'branch', 'store', 'other']
df_global = pd.DataFrame(columns=my_labels)
for file_name in files:
    test_name = file_name
    df = read_histogram(os.path.join("tests/pre_silicon_runs_ml_db",file_name))
    df_sumary, df_count = calculate_values(df)
    display(df_sumary)
    df_global.loc[test_name] = df_sumary['probability'].values.tolist()

plot_stacked_bar(df_global)

df_global.reset_index(inplace=True)
df_global = df_global.rename(columns={"index": "test_name"} )

print(df_global)


df_global.to_csv(results_file)

,InstrKind,probability
0,arithmetic,0.115361
1,branch,0.300828
2,store_counter,0.386302
3,load_counter,0.029231
4,vector_counter,0.002293
5,io_counter,0.000114
6,other_counter,0.165872


ValueError: cannot set a row with mismatched columns

In [ ]:

df = df_global

df_ = df.loc[:, df.columns != 'test_name']
df_ = (df_-df_.min())/(df_.max()-df_.min())
extracted_col = df["test_name"]
df_.insert(0, 'test_name', extracted_col)
df_.set_index('test_name')
df_.to_csv('normalized_df.csv')
df = df_

features = list(df.columns)[1:]
test_column = list(df.columns)[0]

pca_df = get_PCA(df, features, test_column)


# Plot characteristics of the worklaods

In [ ]:
from clustering.distortion import *
from clustering.cluster_score import *
from pca.pca import *

pca_df = get_PCA(df_global,features)
clusters = calculate_elbow(pca_df)[0]
display(f"Optimal number of clusters = {clusters}")

## Clasification based on optimal number of clusters

In [ ]:
from clustering.clustering import *
df_result,kmeans = kmean_cluster(pca_df,2)

# create a DF based on KMEANS prediuction
df_clusters = df_result[['PRED', 'test_name']].copy()
df_clusters.sort_values(by=['test_name'], ascending=True, inplace= True)
pca_df_sorted = pca_df.sort_values(by=['test_name'], ascending=True)
df_clusters["principal component 1"] = pca_df_sorted["principal component 1"].values
df_clusters["principal component 2"] = pca_df_sorted["principal component 2"].values
df_clusters.rename(columns={'PRED': 'cluster'}, inplace=True)
display(df_clusters)
plot_kmeans(pca_df, kmeans)

# Get the test cases closer to centroid

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
X = pd.read_csv("pca.csv", usecols = ['principal component 1','principal component 2'])
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
pca_df = pd.read_csv("pca.csv")
df = pca_df.iloc[closest]
df

In [ ]:

list_tests = df['test_name'].tolist()
df2 = df_global[df_global['test_name'].isin(list_tests)]
df2.plot.barh(x="test_name")
df2


In [ ]:
x = 0
for test in df_global['test_name']:
    if "redis" in test:
        df_global.loc[df_global.index[x], 'Kind'] = int(1)
    else:
        df_global.loc[df_global.index[x], 'Kind'] = int(0)
    x+=1
display(df_global)
df_data = df_global

In [ ]:
df_sorted = df_data.sort_values(by='Kind')
fp_tests = df_sorted[df_sorted['Kind'] == 0]
int_tests = df_sorted[df_sorted['Kind'] == 1]

df = df_result
df_sorted = df.sort_values(by='PRED')
A = df_sorted[df_sorted['PRED'] == 0]
B = df_sorted[df_sorted['PRED'] == 1]

C = normalize(A,B,fp_tests,int_tests)
C_sorted =  C.sort_values(by='test_name', ascending=False)

REAL_sorted = df_data.sort_values(by='test_name', ascending=False)

labels_pred = C_sorted["PRED"].tolist()
labels_pred_ = [eval(i) for i in labels_pred]
labels_pred = labels_pred_

labels_true = REAL_sorted["Kind"].tolist()

df = pd.DataFrame({'labels':C_sorted['test_name'], 'val_pred':labels_pred, 'val_true':labels_true})

print(df)
print(df.loc[~(df['val_pred'] == df['val_true'])])

ax = df.plot.barh(x='labels', y=['val_pred','val_true'],  rot=0, title= "Cluster 0: FP / 1: INT")


scores_result = scores(labels_true,labels_pred)

df = pd.DataFrame({'scores':['adjusted_rand_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score'], 'val':scores_result[:-1]})
ax = df.plot.barh(x='scores', y='val', rot=0, title= "Scores (HIB)")

## Clasification based on DB/AI clusters


In [ ]:
# create a DF based on real known kind
df_clusters_ = df_data[['Kind', 'test_name']].copy()
df_clusters_.sort_values(by=['test_name'], ascending=True, inplace= True)
pca_df_sorted_ = pca_df.sort_values(by=['test_name'], ascending=True)
df_clusters_["principal component 1"] = pca_df_sorted_["principal component 1"].values
df_clusters_["principal component 2"] = pca_df_sorted_["principal component 2"].values
df_clusters_.rename(columns={'Kind': 'cluster'}, inplace=True)
display(df_clusters_)

## Use supervised ML to validate with testing and training data

In [ ]:
# Use known kind based on SPEC documentation
X = df_clusters_[["principal component 1","principal component 2"]]
y = df_clusters_[["cluster"]]

# Use known kind based on KMEANS prediction
#X = df_clusters[["principal component 1","principal component 2"]]
#y = df_clusters[["cluster"]]

accuracy_results = {}

### K-Nearest Neighbors Algorithm better accuracy


In [ ]:

# Use known kind based on KMEANS prediction
X = df_clusters[["principal component 1","principal component 2"]]
y = df_clusters[["cluster"]]

accuracy_results = {}

In [ ]:
# Split into training and test set
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

knn_score = 0

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    knn = KNeighborsClassifier(n_neighbors=7)
    knn.fit(X_train,y_train.values.ravel())
    print(knn.score(X_test, y_test))
    knn_score += knn.score(X_test, y_test)
    

print(f'knn.score average  = {knn_score/10}')
accuracy_results['knn_score'] = knn_score/10

In [ ]:
### Decision Tree Algorithm


In [ ]:
def get_accuracty(X,y,loops,clf):
    p_total = 0
    r_total = 0
    f_total = 0

    for i in range(loops):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
        clf.fit(X_train,y_train)

        # Predict Accuracy Score
        y_pred = clf.predict(X_test)
        p,r,f,s = precision_recall_fscore_support(y_test, y_pred, average='weighted', labels=y_pred,zero_division=1)

        p_total += p
        r_total += r
        f_total += f
    return p_total,r_total,f_total

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support

# Split into training and test set
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

loops = 10
clf = DecisionTreeClassifier()
p_total,r_total,f_total = get_accuracty(X,y,loops,clf)
    
print(f"Precision score {p_total/10}")
print(f"Recall score {r_total/10}")
print(f"F-beta score {f_total/10}")

accuracy_results['dt_precision_score'] = p_total/10
accuracy_results['dt_recall_score'] = r_total/10
accuracy_results['dt_fbeta_score'] = f_total/10

In [ ]:
accuracy_results_df = pd.DataFrame(list(accuracy_results.items()),columns=['algorithm','accuracy'])
display(accuracy_results_df)
accuracy_results_df.plot.barh(x='algorithm', title = 'Accuracy of suppervised ML algorithms with Pre Silicon data');